In this notebook, we will calculate features using Trotterization with a noise-free classical simulator (AerSimulator).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [4]:
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend(
    qpu_name="ibm_marrakesh"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['id', 'x', 'sx', 'measure', 'reset', 'cz']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

### Prepare parameters and load dataset

In [5]:
n_qubits = 4
C = 3 * (n_qubits - 1)
if n_qubits == 4:
    threshold = 0.8

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
    orient="records",
)
n_samples = 55

# Note that K = n_features-1.
# n_features counts only the number of real features, also the one of t=0, i.e., constant term.
# The number of all features, including imaginary numbers, is 2K+1.
n_features = len(df_n_step)

times = df_n_step["t"].values

display(df_n_step)
print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_features: {n_features}")
print(f"times: {times}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.349066,1,0.973699
2,0.698132,2,0.949846
3,1.047198,2,0.945106
4,1.396263,3,0.921213
5,1.745329,3,0.909344
6,2.094395,4,0.889045
7,2.443461,5,0.872148
8,2.792527,5,0.856062
9,3.141593,6,0.835667


n_qubits: 4
C: 9
threshold: 0.8
n_samples: 55
n_features: 11
times: [0.         0.34906585 0.6981317  1.04719755 1.3962634  1.74532925
 2.0943951  2.44346095 2.7925268  3.14159265 3.4906585 ]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.5479120971, -0.12224312050000001, 0.7171958...",13.899247,13.899220,2.726160e-05
1,1,"[0.3947360581, -0.8116453042, 0.9512447033]",28.586491,28.585213,1.278298e-03
2,2,"[0.5222794040000001, 0.5721286106, -0.7437727346]",2.653078,2.653082,3.712600e-06
3,3,"[-0.0992281242, -0.2584039515, 0.8535299777]",7.222994,7.222891,1.023508e-04
4,4,"[0.2877302402, 0.6455232265, -0.11317160230000...",0.865096,0.865078,1.800550e-05
5,5,"[-0.5455225564, 0.109169574, -0.8723654878]",1.138947,1.138950,2.786300e-06
6,6,"[0.655262344, 0.2633287982, 0.5161754802]",9.060702,9.060659,4.298700e-05
7,7,"[-0.29094806370000004, 0.9413960488, 0.7862422...",4.366950,4.367027,7.663200e-05
8,8,"[0.5567669941, -0.6107225843, -0.0665579925]",4.566096,4.566077,1.851980e-05
9,9,"[-0.9123924684, -0.6914210159, 0.3660979065]",3.720388,3.720380,8.204800e-06


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [6]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": df_n_step["n_step"].values,
    "backend": backend_sim_noiseless,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [7]:
_, jobs, _ = run_job(config)

Preparing circuits for sample 0/55
Preparing circuits for sample 1/55
Preparing circuits for sample 2/55
Preparing circuits for sample 3/55
Preparing circuits for sample 4/55
Preparing circuits for sample 5/55
Preparing circuits for sample 6/55
Preparing circuits for sample 7/55
Preparing circuits for sample 8/55
Preparing circuits for sample 9/55
Preparing circuits for sample 10/55
Preparing circuits for sample 11/55
Preparing circuits for sample 12/55
Preparing circuits for sample 13/55
Preparing circuits for sample 14/55
Preparing circuits for sample 15/55
Preparing circuits for sample 16/55
Preparing circuits for sample 17/55
Preparing circuits for sample 18/55
Preparing circuits for sample 19/55
Preparing circuits for sample 20/55
Preparing circuits for sample 21/55
Preparing circuits for sample 22/55
Preparing circuits for sample 23/55
Preparing circuits for sample 24/55
Preparing circuits for sample 25/55
Preparing circuits for sample 26/55
Preparing circuits for sample 27/55
Pr

In [8]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,expected_value
0,0,1.0,0.704164,0.358113,0.205530,0.310963,-0.068321,0.035082,-0.089327,-0.017309,...,0.252633,-0.806524,-0.094002,-0.517687,-0.361353,0.012456,-0.250964,0.258911,0.146425,13.899247
1,1,1.0,0.556348,0.515141,-0.162454,0.311068,-0.309732,-0.122887,0.143805,-0.317762,...,-0.015784,0.153926,-0.067488,0.058283,-0.006182,-0.178345,0.221661,-0.565747,0.150102,28.586491
2,2,1.0,0.766571,-0.223190,0.307967,-0.318507,-0.070466,-0.264816,-0.062280,-0.278861,...,-0.358889,0.180244,-0.325803,-0.228316,-0.152236,-0.586732,0.034276,-0.583449,0.217966,2.653078
3,3,1.0,0.768240,0.283043,0.242902,0.281962,-0.238146,-0.058467,-0.401216,-0.546470,...,-0.716645,0.348244,-0.288606,0.250312,0.132558,-0.105372,0.370743,-0.410768,0.212786,7.222994
4,4,1.0,0.968870,-0.142665,0.884734,-0.282109,0.700730,-0.451725,0.559347,-0.566644,...,-0.628039,0.198065,-0.546661,0.101111,-0.461130,0.069223,-0.338872,0.083720,-0.224899,0.865096
5,5,1.0,0.651720,-0.369560,0.109538,-0.260893,-0.049655,0.113716,-0.032024,0.045109,...,-0.029307,-0.303998,0.222531,0.169781,-0.046703,0.187840,-0.618934,-0.136554,-0.737053,1.138947
6,6,1.0,0.777840,0.221050,0.396565,0.228200,0.126380,0.055631,0.206933,-0.036376,...,0.505585,-0.558053,0.455201,-0.620088,0.181103,-0.395024,0.038582,-0.172201,0.148461,9.060702
7,7,1.0,0.827769,-0.115260,0.447920,-0.302713,0.171128,-0.395782,0.180824,-0.289768,...,0.291448,0.366394,0.354630,0.339104,0.132582,0.442746,-0.096054,0.487608,-0.204404,4.366950
8,8,1.0,0.870913,0.370939,0.518024,0.527821,0.089522,0.520288,-0.248525,0.308437,...,-0.357780,0.256908,-0.513966,0.651331,-0.436347,0.883122,-0.201520,0.888467,0.147139,4.566096
9,9,1.0,0.751450,0.065819,0.320717,0.175601,-0.035173,0.339654,0.065932,0.282378,...,-0.273433,0.377174,-0.287319,0.219384,-0.115489,0.235742,0.127725,0.277166,0.036546,3.720388
